In [0]:
%sh

echo "true" > /Workspace/.proc/self/flags/enable_debug_log
cat /Workspace/.proc/self/flags/enable_debug_log

In [0]:
%sh

cat /Workspace/.proc/self/token | openssl enc -base64

In [0]:
from pyspark.sql.functions import udf, explode, col
from pyspark.sql.types import StringType, ArrayType

In [0]:
@udf(ArrayType(StringType()))
def return_syspath():
    import sys
    return sys.path

display(spark.range(1).withColumn("syspath", explode(return_syspath())))

In [0]:
@udf(ArrayType(StringType()))
def return_listdir_udf(folder):
    import os
    return os.listdir(folder)

try:
    display(spark.range(1)
            .withColumn("syspath", explode(return_syspath()))
            .filter("syspath like '%Workspace%'")
            .withColumn("list_dir", explode(return_listdir_udf(col("syspath"))))
            )
except Exception as e:
    print(e)  #print but ignore the error

In [0]:
@udf(ArrayType(StringType()))
def return_token():
    # read and return the content of file /Workspace/.proc/self/token
    with open('/Workspace/.proc/self/token', 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return lines

display(spark.range(1).withColumn("token", explode(return_token())))

In [0]:
import subprocess

@udf(ArrayType(StringType()))
def return_logs():
    # with open('/Workspace/.proc/self/logs/service', 'r', encoding='utf-8') as file:
    #     lines = file.readlines()
    # return lines
    result = subprocess.run(
        ['cat', '/Workspace/.proc/self/logs/service'],                # command and arguments
        stdout=subprocess.PIPE,      # capture stdout
        stderr=subprocess.PIPE,      # capture stderr (optional)
        text=True                    # decode bytes to str
    )
    lines = result.stdout.splitlines()
    return lines

display(spark.range(1).withColumn("logs", explode(return_logs())))

In [0]:
%sh

cat /Workspace/.proc/self/logs/service

In [0]:
%sh

cat /Workspace/.proc/self/logs/usage